In [50]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    read_sql_query, 
    DataFrame,
    ExcelWriter
)
from enum import Enum
import openpyxl




In [48]:
class Period(Enum):
    date = '2021-10-01'
    wb=''
    date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query_presenceQty = f'''
SELECT 
    a.*, COUNT(*) AS qte_pres_by_creator
FROM
    (SELECT 
            dgs.created_by,
            au.email AS email,
            au.username
    FROM
        caris_db.dream_group_session dgs
    LEFT JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
    LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
GROUP BY a.created_by
'''

query_beneficiariesQty = f'''
SELECT *, COUNT(*)AS qte_by_creator FROM
    (SELECT
        dgs.id AS id_session,
            topic,
            date,
            dga.id_patient AS id_patient,
            dgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username,
            b.first_name,
            b.last_name,
            p.patient_code as dreams_code
    FROM
        caris_db.dream_group_session dgs
    INNER JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
        LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    left join beneficiary b on dga.id_patient=b.id_patient
    left join patient p on p.id=dga.id_patient
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
            group by a.id_patient, a.username
'''

pres_byAgent= read_sql_query(query_presenceQty, engine, parse_dates=True)
ben_byAgent = read_sql_query(query_beneficiariesQty, engine, parse_dates=True)


# close the pool of connection
engine.dispose()


True

In [5]:
pres_byAgent.rename(columns={'qte_pres_by_creator':'nombre_pres'},inplace=True)
ben_byAgent.rename(columns={'qte_by_creator':'nombre_pres'},inplace=True)

In [11]:
print('Nombre de presence totale: ',pres_byAgent.nombre_pres.sum())
print('Nombre de presence totale: ',ben_byAgent.nombre_pres.sum())
print('Nombre de filles: ',ben_byAgent.id_patient.count())

Nombre de presence totale:  9744
Nombre de presence totale:  9744
Nombre de filles:  605


# First Component

In [19]:
pres_byAgent = pres_byAgent[['username','email','nombre_pres']]
pres_byAgent


,username,email,nombre_pres
0,johanejules,johane.jules@carisfoundationintl.org,16
1,fbyron,byronfabrice9@gmail.com,512
2,billyjolicoeur,billyjolicoeur95@gmail.com,291
3,FredlyEliona,elionafredly7@gmail.com,132
4,dchelot,blonnichel@yahoo.fr,912
5,pggd1234,gerald.poteau@carisfoundationintl.org,532
6,Armando,diegoarmando.dorlean@carisfoundationintl.org,1535
7,Ernstbelabre,ernst.belabre@carisfoundationintl.org,2346
8,France14,francenelstjean.saed@yahoo.com,2036
9,despeignes,roselord.despeignes@carisfoundationintl.org,736


# Component 2

In [56]:
ben_table = ben_byAgent.pivot_table(values='id_patient',index='username',columns='commune',aggfunc='count',fill_value=0,margins=True, margins_name='Total')
ben_table

commune,Cap-Haïtien,Desdunes,Dessalines,Kenscoff,Liancourt,Port-au-Prince,Pétionville,Saint-Marc,Tabarre,Verrettes,Total
username,,,,,,,,,,,
Armando,0,0,0,0,0,0,0,8,0,50,58
BonWoodRidchard,0,0,0,0,0,22,14,0,4,0,40
Ernstbelabre,0,0,19,0,0,0,0,118,0,0,137
France14,0,18,124,0,0,0,0,0,0,0,142
FredlyEliona,0,0,0,0,0,0,11,0,0,0,11
billyjolicoeur,0,0,0,0,0,9,0,0,6,0,15
dchelot,0,0,0,0,25,0,0,44,0,0,69
despeignes,0,0,0,0,0,0,0,3,0,40,43
fbyron,26,0,0,8,0,4,20,0,0,0,58


# 3 component 

In [46]:
curriculum_result = DataFrame(
    [
        {'Nombre de fille': ben_byAgent.id_patient.count(),
        'Nombre de presence': pres_byAgent.nombre_pres.sum()}
    ]
)
curriculum_result

,Nombre de fille,Nombre de presence
0,605,9744


# Report

In [58]:
all_report = ExcelWriter(f"presence_curriculum.xlsx",engine="openpyxl")
curriculum_result.to_excel(all_report,sheet_name="resultat",index=False,na_rep="")
pres_byAgent.to_excel(all_report,sheet_name="presence_par_agent",index=False,na_rep="")
ben_table.to_excel(all_report,sheet_name="beneficiaires_par_communes",index=True,na_rep="")
all_report.save()